In [1]:
from loader import load_reviews_to_list
from text_search import *
from spatial_search import *

In [2]:
path = 'Data\\Restaurants_London_England.tsv'
reviews, borders = load_reviews_to_list(path)

In [3]:
x_list, y_list = create_grid(borders,50)

In [4]:
grid = add_restaurants_to_grid(x_list, y_list, reviews)

In [5]:
print_grid_info(borders, grid)

bounds: 50.546856 55.95297 -4.243601 1.387973
widths: 5.406114000000002 5.631574
0 2 1
4 29 1
5 36 5
6 32 1
6 34 1
6 39 1
6 49 1
7 33 3
7 34 4
7 35 49
7 36 32
7 37 18
7 38 4
8 15 1
8 32 1
8 33 24
8 34 149
8 35 1203
8 36 4894
8 37 759
8 38 71
8 39 1
8 40 1
9 33 6
9 34 54
9 35 461
9 36 1999
9 37 866
9 38 78
9 39 2
9 40 2
9 43 3
10 18 1
10 34 2
10 35 5
10 36 87
10 37 36
10 38 4
10 39 1
12 34 1
12 42 1
13 20 1
13 49 1
17 20 1
19 27 1
23 25 1
26 17 1
27 17 1
29 13 1
29 24 1
49 0 1
49 9 1


In [9]:
q = [51,51.20,-4.243601,1.387973]
results_grid, grid_time = spaSearchGrid(q, x_list, y_list, grid, reviews)
results_raw, raw_time = spaSearchRaw(q, reviews)

X: 4 6
X_LIST: [50.97934512, 51.0874674] [51.19558968, 51.30371196]
Y: 0 49
Y_LIST: [-4.243601, -4.13096952] [1.2753415199999996, 1.387973]


In [10]:
print('spaSearchRaw:', len(results_raw), 'results, cost:', raw_time, 'seconds')
for r in results_raw:
    print(r['review_id'])
    print('location:', r['x'], ',', r['y'])
    print('tags:', r['tags'], '\n')

spaSearchRaw: 6 results, cost: 0.0029888153076171875 seconds
Restaurant_Review-g186338-d3318593-Reviews-Jamie_Oliver_s_Restaurant-London_England.html
location: 51.159298 , -0.172481
tags: ['international'] 

Restaurant_Review-g186338-d3381895-Reviews-Joes_Kitchen_Coffee_House-London_England.html
location: 51.161 , -0.172959
tags: ['central european'] 

Restaurant_Review-g186338-d2343541-Reviews-Garfunkel_s_at_Gatwick_Aiport-London_England.html
location: 51.160873 , -0.179375
tags: ['european', 'breakfast/brunch', 'late night'] 

Restaurant_Review-g186338-d3979097-Reviews-BB_Bakery-London_England.html
location: 51.008526 , -0.936142
tags: ['french'] 

Restaurant_Review-g186338-d4046898-Reviews-Jamie_s_italian-London_England.html
location: 51.16089 , -0.176477
tags: ['italian'] 

Restaurant_Review-g186338-d2375022-Reviews-Garfunkel_s-London_England.html
location: 51.160877 , -0.17403
tags: ['english'] 



In [11]:
print('spaSearchGrid:', len(results_grid), 'results, cost:', grid_time, 'seconds')
for r in results_grid:
    print(r['review_id'])
    print('location:', r['x'], ',', r['y'])
    print('tags:', r['tags'], '\n')

spaSearchGrid: 6 results, cost: 0.0009996891021728516 seconds
Restaurant_Review-g186338-d3979097-Reviews-BB_Bakery-London_England.html
location: 51.008526 , -0.936142
tags: ['french'] 

Restaurant_Review-g186338-d3318593-Reviews-Jamie_Oliver_s_Restaurant-London_England.html
location: 51.159298 , -0.172481
tags: ['international'] 

Restaurant_Review-g186338-d3381895-Reviews-Joes_Kitchen_Coffee_House-London_England.html
location: 51.161 , -0.172959
tags: ['central european'] 

Restaurant_Review-g186338-d2343541-Reviews-Garfunkel_s_at_Gatwick_Aiport-London_England.html
location: 51.160873 , -0.179375
tags: ['european', 'breakfast/brunch', 'late night'] 

Restaurant_Review-g186338-d4046898-Reviews-Jamie_s_italian-London_England.html
location: 51.16089 , -0.176477
tags: ['italian'] 

Restaurant_Review-g186338-d2375022-Reviews-Garfunkel_s-London_England.html
location: 51.160877 , -0.17403
tags: ['english'] 

